In [1]:
import math
import re
import nltk
import string
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
"""documents = [
    "Selenium is a portable framework for testing web applications",
    "Beautiful Soup is useful for web scraping",
    "It is a python package for parsing the pages"
]"""
documents = [
    "dictionary structure compression",
    "block structure sorting",
    "block expensive structure"
]

In [3]:
def preprocess(txt):
    s = txt
    s = s.lower()
    s = re.sub(r'\d+', '',s)
    s = s.replace('/[^A-Za-z0-9]/g', '')
    s = s.strip()
    words = word_tokenize(s)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return words

def findoccurences(txt,word):
    txt = txt.replace('/[^A-Za-z0-9]/g', '')
    txt = txt.replace('  ',' ')
    txt = txt.lower()
    txt_words = txt.strip().split()
    
    word_count = 0
    word_positions = []
    
    for i in range(len(txt_words)):
        if word == txt_words[i]:
            word_count += 1
            word_positions.append(i)
            
    return (word_count, word_positions)

In [4]:
inverted_index = {}
for (i,doc) in enumerate(documents):
    words = preprocess(doc)
    
    for word in words:
        if word not in inverted_index:
            inverted_index[word] = []
            
        occurence_count, occurence_pos_list = findoccurences(doc, word)
        
        inverted_index[word].append(((i+1), occurence_count, occurence_pos_list))

print('Inverted Index: ')
for item in inverted_index.items():
    print(item)

Inverted Index: 
('dictionary', [(1, 1, [0])])
('structure', [(1, 1, [1]), (2, 1, [1]), (3, 1, [2])])
('compression', [(1, 1, [2])])
('block', [(2, 1, [0]), (3, 1, [0])])
('sorting', [(2, 1, [2])])
('expensive', [(3, 1, [1])])


In [5]:
def eliasGammaEncoding(n):
    if n == 0:
        return "0"

    num = binary(n)
    
    num = ('0' * (len(num) -1)) + num
    
    return num

In [6]:
def eliasDeltaEncoding(n):
    if n == 0:
        return "0"
    num1 = 1 + int(math.log2(n))
    num1 = eliasGammaEncoding(num1)
    
    num2 = binary(n)
    num2 = str(num2)[1:]
    
    num = num1 + num2
    
    return num

def binary(n) :
    
    num = bin(n)
    
    num = num[2:]
    
    return num

def indexCompression(inverted_index, encoding_scheme):
    
    compression_map = {}
    
    for word_indices in inverted_index.values():
        for word_index in word_indices:
            i, count, positions = word_index
            arr = [i, count] + positions
            
            for n in arr:
                if n not in compression_map:
                    if encoding_scheme == 'ELIAS_DELTA' :
                        compression_map[n] = eliasDeltaEncoding(n)
                    elif encoding_scheme == 'VARIABLE_BYTE' :
                        compression_map[n] = variableByteEncoding(n)
                    elif encoding_scheme == 'BINARY_ENCODING' :
                        compression_map[n] = binary(n)
                        
    return compression_map

bin_encoding_compression_map = indexCompression(inverted_index, 'BINARY_ENCODING')

print("Binary Encoding Map: ")
for item in bin_encoding_compression_map.items():
    print(item)

Binary Encoding Map: 
(1, '1')
(0, '0')
(2, '10')
(3, '11')


In [7]:
elias_delta_compression_map = indexCompression(inverted_index, 'ELIAS_DELTA')

print("Elias Delta Encoding Map: ")
for item in elias_delta_compression_map.items():
    print(item)

Elias Delta Encoding Map: 
(1, '1')
(0, '0')
(2, '0100')
(3, '0101')
